In [1]:
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

## Connect to Snowflake

In [2]:
import snowflake.connector
from snowflake_credentials import snowflake_params

conn = snowflake.connector.connect(
    user=snowflake_params['user'],
    password=snowflake_params['password'],
    account=snowflake_params['account'],
    warehouse=snowflake_params['warehouse'],
    database=snowflake_params['database'],
    schema=snowflake_params['schema'],
    role=snowflake_params['role']
)

cur = conn.cursor()

## Table Data Preview

### Yelp Businesses Table

In [11]:
businesses_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_businesses
                               ''').fetch_pandas_all()

print(f'There are {businesses_count['COUNT(*)'][0]:,} records in the Yelp businesses table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_businesses 
                 LIMIT 5
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 1: Yelp Businesses Table Preview</span>').set_table_styles(styles))

There are 150,346 records in the Yelp businesses table.


,NAME,BUSINESS_ID,IS_OPEN,REVIEW_COUNT,STARS,CATEGORIES,STATE,CITY,POSTAL_CODE,LATITUDE,LONGITUDE
0,"Abby Rappoport, LAC, CMQ",Pns2l4eNsfO8kk83dixA6A,0,7,5.000000,"Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists",CA,Santa Barbara,93101,34.426679,-119.711197
1,The UPS Store,mpf3x-BjTdTEA3yCZrAYPw,1,15,3.000000,"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services",MO,Affton,63123,38.551126,-90.335695
2,Target,tUFrWirKiKi_TAnsVWINQQ,0,22,3.500000,"Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores",AZ,Tucson,85711,32.223236,-110.880452
3,St Honore Pastries,MTSW4McQd7CbVtyjqoe9mw,1,80,4.000000,"Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",PA,Philadelphia,19107,39.955505,-75.155564
4,Perkiomen Valley Brewery,mWMc6_wTdE0EUBKIGXDVfA,1,13,4.500000,"Brewpubs, Breweries, Food",PA,Green Lane,18054,40.338183,-75.471659


### Yelp Reviews Table

In [15]:
reviews_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_reviews
                               ''').fetch_pandas_all()

print(f'There are {reviews_count['COUNT(*)'][0]:,} records in the Yelp reviews table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_reviews 
                 LIMIT 1
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 2: Yelp Reviews Table Preview</span>').set_table_styles(styles))

There are 6,990,280 records in the Yelp reviews table.


,BUSINESS_ID,REVIEW_ID,USER_ID,REVIEW_DATE,USER_REVIEW,STARS,COOL_VOTES,FUNNY_VOTES,USEFUL_VOTES
0,iRIHK8-EwpeffwvoO4nzIA,oXpWjOG2rue-2bHbjvDZIQ,1uHAAjOX18Px4OibCMI3pg,2018-10-04,"Awesome spot. This is one of my favorite places in St. Louis. Its best to go on Saturdays when the bands are there playing. The food is great and restaurant atmosphere is great. This visit we had our beers and tasted a lot of the menu while the band was playing. Some of the stand out items we tastes were the voodoo shrimp, the crawfish mac n cheese, the oysters, and blackened red snapper tacos. The sauce on the voodoo shrimp were my favorite thing all day. The shrimp are lightly fried with a spicy flavorful sauce. The crawfish mac n cheese had plenty of cheese and crawfish tails to be really filling. The oysters served were large and fresh. The blackened red snapper tacoses were seasoned really good and hit the spot.",5,0,0,0


### Yelp Users Table

In [16]:
users_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_users
                               ''').fetch_pandas_all()

print(f'There are {users_count['COUNT(*)'][0]:,} records in the Yelp users table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_users 
                 LIMIT 5
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 3: Yelp Users Table Preview</span>').set_table_styles(styles))

There are 1,987,897 records in the Yelp users table.


,FIRST_NAME,USER_ID,YELPING_SINCE,REVIEW_COUNT,AVG_STARS,FANS,COOL_COMPLIMENTS,CUTE_COMPLIMENTS,FUNNY_COMPLIMENTS,HOT_COMPLIMENTS,LIST_COMPLIMENTS,MORE_COMPLIMENTS,NOTE_COMPLIMENTS,PHOTO_COMPLIMENTS,PLAIN_COMPLIMENTS,PROFILE_COMPLIMENTS,WRITER_COMPLIMENTS,COOL_VOTES,ELITE,FUNNY_VOTES,USEFUL_VOTES
0,Thomas,dXZLLcy6klelF1O1wL_vzw,2015-07-24,11,4.450000,1,0,0,0,0,0,0,0,0,0,0,0,0,,0,2
1,Jeremias JMan,eaiIXArF07R0MGtu4h-6kg,2015-10-29,19,4.420000,1,0,0,0,0,0,0,0,0,0,0,0,2,,9,9
2,Dolly,T_Fsemvyy8IsoB90-sS0sQ,2014-05-05,48,4.320000,5,1,1,1,2,0,0,0,0,1,0,0,13,,10,38
3,Ellyse,EaRd_4nFnPs8Fei-H2ZmVA,2013-07-18,427,4.310000,61,31,0,31,5,0,2,8,17,15,0,23,420,"2017,2018,2019,20,20",108,823
4,Julie,yqpqFUXIkKBYP5QQ-5jZpA,2013-06-13,48,3.340000,1,1,0,1,0,0,0,1,0,0,0,0,17,,28,62


### Yelp Tips Table

In [17]:
tips_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_tips
                               ''').fetch_pandas_all()

print(f'There are {tips_count['COUNT(*)'][0]:,} records in the Yelp tips table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_tips 
                 LIMIT 5
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 4: Yelp Tips Table Preview</span>').set_table_styles(styles))

There are 908,915 records in the Yelp tips table.


,BUSINESS_ID,USER_ID,DATE,TIP,COMPLIMENT_COUNT
0,WnHG_obpmHazbngTdu570Q,_Wkf2RNDnBuqMOpzSqmZXw,2015-08-14,Only a couple more weeks,0
1,c-CAcdH2-12g1sQUQfy5xw,xnuEEuMfvCNLFMtaZf7asw,2016-03-20,Get here early!!,0
2,wj8XtPyuREj8_0GQz3LZ6w,W0DJOPsSwcAj0uqCJG8iLw,2015-08-14,If you have not tried this place what are you waiting for? Just GO NOW!,0
3,l8vwslfqKq1yxlmgVi-M0Q,AED6tP0v4aMPd9-YtjPcPg,2015-05-11,Check out the third floor lounge for nice city views and a relaxed place to have a couple cocktails. It's tough to find a good lounge in St. Louis!,0
4,KZe5kpIHWVuJl6WgNYzceg,9UZlB3fHVe2JBvPw4kRb5g,2012-03-03,Pancakes with Breakfast Trick?,0


## Category-Level Analysis

### Count of Categories

In [18]:
# Count of categories in Yelp dataset
df = cur.execute('''
                 WITH cat_count AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT 
                        TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT COUNT(DISTINCT category) AS num_of_categories
                FROM cat_count;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 5: Count of Categories</span>').set_table_styles(styles))

,NUM_OF_CATEGORIES
0,1311


### Top 20 Categories by Number of Businesses

In [19]:
# Top 20 categories of all businesses on Yelp.
df = cur.execute('''
                 WITH cat AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_rating
                FROM cat
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 6: Top 20 Categories by Business Count</span>').set_table_styles(styles))

,CATEGORY,NUM_OF_BUSINESSES,AVG_RATING
0,Restaurants,52268,3.520000
1,Food,27781,3.670000
2,Shopping,24395,3.620000
3,Home Services,14356,3.460000
4,Beauty & Spas,14292,3.810000
5,Nightlife,12281,3.660000
6,Health & Medical,11890,3.630000
7,Local Services,11198,3.620000
8,Bars,11065,3.650000
9,Automotive,10773,3.510000


### Top 20 Categories by Number of Reviews

In [20]:
# Top 20 categories by number of reviews
df = cur.execute('''
                 WITH cat AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_rating
                FROM cat
                GROUP BY 1  
                ORDER BY 2 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 7: Top 20 Categories by Review Volume</span>').set_table_styles(styles))

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_RATING
0,Restaurants,4561279,52268,3.520000
1,Food,1752281,27781,3.670000
2,Nightlife,1488163,12281,3.660000
3,Bars,1406415,11065,3.650000
4,American (Traditional),976483,8139,3.400000
5,American (New),950560,6097,3.590000
6,Breakfast & Brunch,839467,6239,3.570000
7,Sandwiches,669139,8366,3.540000
8,Seafood,600183,3539,3.670000
9,Event Planning & Services,591442,9895,3.690000


### Top 20 Categories by Average Star Rating (Min 1000 Reviews)

In [22]:
# Top 20 categories by average star rating (minimum 1000 reviews)
df = cur.execute('''
                 WITH cat AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_rating
                FROM cat
                GROUP BY 1  
                HAVING SUM(review_count) >= 1000
                ORDER BY 4 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 8: Top 20 Categories by Average Rating (Min 1000 Reviews)</span>').set_table_styles(styles))

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_RATING
0,Art Tours,1962,18,4.860000
1,Bike tours,1714,32,4.750000
2,Sugaring,1549,39,4.690000
3,Reiki,3048,201,4.680000
4,Photo Booth Rentals,1618,92,4.680000
5,Beer Tours,2949,51,4.650000
6,Meditation Centers,1451,99,4.630000
7,Barre Classes,2926,158,4.610000
8,DJs,2276,127,4.570000
9,Personal Chefs,4552,116,4.570000


### Bottom 20 Categories by Average Star Rating (Min 1000 Reviews)

In [23]:
# Bottom 20 categories by average star rating (minimum 1000 reviews)
df = cur.execute('''
                 WITH cat AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_rating
                FROM cat
                GROUP BY 1  
                HAVING SUM(review_count) >= 1000
                ORDER BY 4
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 9: Bottom 20 Categories by Average Rating (Min 1000 Reviews)</span>').set_table_styles(styles))

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_RATING
0,Television Service Providers,4253,159,2.050000
1,University Housing,2630,122,2.330000
2,Internet Service Providers,7947,356,2.340000
3,Property Management,12282,759,2.500000
4,Post Offices,5284,327,2.560000
5,Apartments,28510,1921,2.580000
6,Parking,9327,243,2.610000
7,Truck Rental,3633,206,2.610000
8,Utilities,2276,102,2.630000
9,Fast Food,224529,6472,2.630000


### Overrated Categories

In [24]:
# Categories with high stars and low engagement
df = cur.execute('''
                 WITH cat AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_rating
                FROM cat
                GROUP BY 1  
                ORDER BY 4 DESC, 2
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 10: Categories with High Average Rating and Low Review Volume</span>').set_table_styles(styles))

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_RATING
0,Metal Detector Services,6,1,5.000000
1,Patent Law,6,1,5.000000
2,Karaoke Rental,7,1,5.000000
3,Circus Schools,9,1,5.000000
4,Bubble Soccer,13,1,5.000000
5,Mohels,14,1,5.000000
6,Art Consultants,15,2,5.000000
7,Calligraphy,16,2,5.000000
8,Water Suppliers,17,1,5.000000
9,Silent Disco,37,2,5.000000


## Rating Distribution Overview

### Total Review Volume for Each Star Rating

In [26]:
# Count of reviews for each star value 1-5
df = cur.execute('''
                 SELECT '1 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 1
                 UNION
                 SELECT '2 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 2
                 UNION
                 SELECT '3 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 3
                 UNION
                 SELECT '4 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 4
                 UNION
                 SELECT '5 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 5
                 ORDER BY 1;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 11: Review Volume by Star Rating</span>').set_table_styles(styles))

,REVIEW_RATING,NUM_REVIEWS
0,1 Star,1069561
1,2 Star,544240
2,3 Star,691934
3,4 Star,1452918
4,5 Star,3231627


### Counts of Each Star Rating per Business

In [28]:
# Count of reviews with each star value by business
df = cur.execute('''
                 SELECT 
                    business_id, 
                    SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                    SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                    SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                    SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                    SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                    COUNT(*) AS total_reviews,
                    SUM(stars) AS total_stars
                 FROM yelp_reviews
                 GROUP BY 1
                 LIMIT 10;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 12: Star Rating Breakdown by Business</span>').set_table_styles(styles))

,BUSINESS_ID,ONE_STAR_REVIEWS,TWO_STAR_REVIEWS,THREE_STAR_REVIEWS,FOUR_STAR_REVIEWS,FIVE_STAR_REVIEWS,TOTAL_REVIEWS,TOTAL_STARS
0,7ATYjTIgM3jUlt4UM3IypQ,1,3,2,21,124,151,717
1,CLEWowfkj-wKYJlQDqT1aw,7,5,1,4,38,55,226
2,oBhJuukGRqPVvYBfTkhuZA,33,42,68,140,118,401,1471
3,pUVRCYL8rT4I5Ry5FYkRsA,7,3,5,21,70,106,462
4,sLgnx_WFCjEoPsS6NwU70Q,20,28,37,47,138,270,1065
5,FHNIvNgh3fS7VZQq2Y3dsA,15,62,140,266,368,851,3463
6,gGyqnAlpFrka_qzpO7j4lQ,16,12,43,182,276,529,2277
7,Dv6RfXLYe1atjgz3Xf4GGw,33,53,112,377,541,1116,4688
8,s3FRhtaNU8dt-mKkLKUQaQ,47,41,42,62,22,214,613
9,hy5GpGXAna-5qrb3zNub6g,42,31,28,37,41,179,541


### "True" Average Star Rating & Percent of 1 & 5 Star Reviews

In [29]:
# True star average for each business and percentage of five star and one star reviews to total reviews
df = cur.execute('''
                WITH cte AS (
                    SELECT 
                        business_id, 
                        SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                        SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                        SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                        SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                        SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                        COUNT(*) AS total_reviews,
                        SUM(stars) AS total_stars,
                        ROUND(AVG(stars), 2) AS true_avg_rating
                    FROM yelp_reviews
                    GROUP BY 1
                 )

                SELECT 
                    yb.name, yb.city, yb.state, total_reviews, total_stars,
                    true_avg_rating,
                    yb.stars AS rounded_avg_rating,
                    ROUND(five_star_reviews / total_reviews * 100, 2) AS five_star_pct,
                    ROUND(one_star_reviews / total_reviews * 100, 2) AS one_star_pct,
                FROM cte
                JOIN yelp_businesses yb
                    ON cte.business_id = yb.business_id
                LIMIT 20;
                ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 13: "True" Average Star Rating & Percentage of 1 & 5 Star Reviews</span>').set_table_styles(styles))

,NAME,CITY,STATE,TOTAL_REVIEWS,TOTAL_STARS,TRUE_AVG_RATING,ROUNDED_AVG_RATING,FIVE_STAR_PCT,ONE_STAR_PCT
0,The Gables Apartments,Greenwood,IN,12,16,1.33,1.500000,0.00,83.33
1,Schiano's Pizza,Warminster,PA,29,83,2.86,3.000000,31.03,41.38
2,Caliber Collision,Largo,FL,10,34,3.40,3.500000,50.00,30.00
3,Gail Marcus,Philadelphia,PA,9,45,5.00,5.000000,100.00,0.00
4,Employ Health,Nashville,TN,10,50,5.00,5.000000,100.00,0.00
5,Value Vet,Nashville,TN,51,184,3.61,3.500000,47.06,15.69
6,America's Mattress of Tucson,Tucson,AZ,7,35,5.00,5.000000,100.00,0.00
7,Terry House Bed & Breakfast,New Castle,DE,5,22,4.40,4.500000,40.00,0.00
8,Little Caesers Pizza,Souderton,PA,5,17,3.40,3.500000,60.00,40.00
9,Piano Gastrolounge,Santa Barbara,CA,24,78,3.25,3.500000,25.00,29.17


## Time Series of Review Volume & Star Metrics

### Monthly Review Volume & Star Metrics

In [30]:
# Months with the highest total reviews and highest average star rating
df = cur.execute('''
                SELECT 
                    MONTH(review_date) AS month,
                    SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                    SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                    SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                    SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                    SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                    COUNT(*) AS total_reviews,
                    SUM(stars) AS total_stars,
                    ROUND(AVG(stars), 2) avg_rating
                FROM yelp_reviews
                GROUP BY 1
                ORDER BY 1;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 14: Monthly Review Volume & Star Metrics</span>').set_table_styles(styles))

# Rename columns for readability in charts
df.rename(columns={'ONE_STAR_REVIEWS': '1 Star', 'TWO_STAR_REVIEWS': '2 Star', 'THREE_STAR_REVIEWS': '3 Star', 'FOUR_STAR_REVIEWS': '4 Star', 'FIVE_STAR_REVIEWS': '5 Star', 'TOTAL_REVIEWS': 'Total Reviews',
                   'AVG_RATING': 'Average Rating'}, inplace=True)

# Generate line chart for review count for each star value by month
fig = px.line(
    df,
    x='MONTH',
    y=[
        '1 Star',
        '2 Star',
        '3 Star',
        '4 Star',
        '5 Star'
    ],
    labels={
        'value': 'Review Count',
        'variable': 'Star Rating',
        'MONTH': 'Month'
    },
    title='Figure 1: Monthly Yelp Review Star Ratings Trends'
)
fig.show()

# Generate bar chart for monthly review volume by month
fig = px.bar(
    df, 
    x='MONTH', 
    y='Total Reviews', 
    color='Total Reviews', 
    labels={'MONTH': 'Month', 'Total Reviews': 'Review Count'}, 
    title='Figure 2: Monthly Yelp Review Volume Trends'
)
fig.show()

,MONTH,ONE_STAR_REVIEWS,TWO_STAR_REVIEWS,THREE_STAR_REVIEWS,FOUR_STAR_REVIEWS,FIVE_STAR_REVIEWS,TOTAL_REVIEWS,TOTAL_STARS,AVG_RATING
0,1,86852,46047,61474,131145,279014,604532,2283018,3.78
1,2,80051,42503,54620,116561,250390,544125,2047111,3.76
2,3,87976,46933,60527,127924,275195,598555,2251094,3.76
3,4,82676,43029,55083,116372,254311,551471,2071026,3.76
4,5,91994,46318,58076,120991,269196,586575,2188802,3.73
5,6,93282,46701,58791,123158,279805,601737,2254714,3.75
6,7,100135,51170,64669,135171,303482,654627,2454576,3.75
7,8,98608,49976,62289,129710,295801,636384,2383272,3.75
8,9,89705,44552,55136,114482,261499,565374,2109640,3.73
9,10,89243,44267,55856,115923,266520,571809,2141637,3.75


### Monthly Review Volume & Star Metrics by Year

In [32]:
# Total reviews by each star amount and the average star review for each month in every year
df = cur.execute('''
                SELECT 
                    YEAR(review_date) AS year,
                    MONTH(review_date) AS month,
                    SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                    SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                    SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                    SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                    SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                    COUNT(*) AS total_reviews,
                    SUM(stars) AS total_stars,
                    ROUND(AVG(stars), 2) avg_rating
                FROM yelp_reviews
                GROUP BY 1, 2
                ORDER BY 1, 2;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(5), df.tail(5)])

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 15: Monthly Review Volume & Star Metrics by Year</span>').set_table_styles(styles))

# Rename columns for readability in charts
df.rename(columns={'ONE_STAR_REVIEWS': '1 Star', 'TWO_STAR_REVIEWS': '2 Star', 'THREE_STAR_REVIEWS': '3 Star', 'FOUR_STAR_REVIEWS': '4 Star', 'FIVE_STAR_REVIEWS': '5 Star', 'TOTAL_REVIEWS': 'Total Reviews',
                   'AVG_RATING': 'Average Rating'}, inplace=True)

# Create a date column from year and month
df['date'] = df['MONTH'].astype(str).str.zfill(2) + '/' + df['YEAR'].astype(str)

# Melt into long format for counts of reviews with each star value
long_df = df.melt(
    id_vars=['date'],
    value_vars=['1 Star','2 Star','3 Star','4 Star','5 Star'],
    var_name='star',
    value_name='count'
)

# Plot time series of star value counts by year and month
fig = px.line(
    long_df,
    x='date',
    y='count',
    color='star',
    labels={
        'date':'Month-Year',
        'count':'Review Count',
        'star':'Star Rating'
    },
    title='Figure 3: Monthly Yelp Star Rating Counts by Year'
)
fig.show()

# Melt into long format for total review counts for each month
long_df = df.melt(
    id_vars=['date'],
    value_vars=['Total Reviews'],
    var_name='reviews',
    value_name='count'
)

# Plot time series of total reviews by year and month
fig = px.line(
    long_df, 
    x='date', 
    y='count', 
    labels={
        'date': 'Month-Year',
        'count': 'Review Count'
        }, 
    title='Figure 4: Monthly Yelp Review Volume by Year'
)
fig.show()

# Melt into long format for average star rating for each month
long_df = df.melt(
    id_vars=['date'],
    value_vars=['Average Rating'],
    var_name='avg_rating',
    value_name='count'
)

# Plot time series of average star rating by year and month
fig = px.line(
    long_df, 
    x='date', 
    y='count', 
    labels={
        'date': 'Month-Year',
        'count': 'Average Star Rating'
        }, 
    title='Figure 5: Monthly Average Yelp Star Rating by Year'
)
fig.show()

,YEAR,MONTH,ONE_STAR_REVIEWS,TWO_STAR_REVIEWS,THREE_STAR_REVIEWS,FOUR_STAR_REVIEWS,FIVE_STAR_REVIEWS,TOTAL_REVIEWS,TOTAL_STARS,AVG_RATING
0,2005,2,0,0,0,2,1,3,13,4.33
1,2005,3,5,8,18,20,23,74,270,3.65
2,2005,4,1,0,4,10,11,26,108,4.15
3,2005,5,1,7,16,42,42,108,441,4.08
4,2005,6,2,2,4,15,15,38,153,4.03
199,2021,9,10470,3295,3348,5775,23509,46397,167749,3.62
200,2021,10,10539,3510,3567,6507,25181,49304,180193,3.65
201,2021,11,9426,3180,3193,6290,23135,45224,166200,3.68
202,2021,12,10175,3219,3601,6768,24716,48479,178068,3.67
203,2022,1,6607,1959,2183,4351,16565,31665,117303,3.70


### Quarterly Review Volume & Star Metrics

In [33]:
# Reviews by quarter
df = cur.execute('''
                SELECT 
                    QUARTER(review_date) AS quarter,
                    SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                    SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                    SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                    SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                    SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                    COUNT(*) AS total_reviews,
                    SUM(stars) AS total_stars,
                    ROUND(AVG(stars), 2) avg_rating
                FROM yelp_reviews
                GROUP BY 1
                ORDER BY 1;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 16: Quarterly Review Volume & Star Metrics</span>').set_table_styles(styles))

# Rename columns for readability in charts
df.rename(columns={'ONE_STAR_REVIEWS': '1 Star', 'TWO_STAR_REVIEWS': '2 Star', 'THREE_STAR_REVIEWS': '3 Star', 'FOUR_STAR_REVIEWS': '4 Star', 'FIVE_STAR_REVIEWS': '5 Star', 'TOTAL_REVIEWS': 'Total Reviews',
                   'AVG_RATING': 'Average Rating'}, inplace=True)

# Generate line chart for review count for each star value by quarter
fig = px.line(
    df,
    x='QUARTER',
    y=[
        '1 Star',
        '2 Star',
        '3 Star',
        '4 Star',
        '5 Star'
    ],
    labels={
        'value': 'Review Count',
        'variable': 'Star Rating',
        'QUARTER': 'Quarter'
    },
    title='Figure 6: Quarterly Yelp Review Star Rating Trends'
)
fig.update_xaxes(dtick=1)
fig.show()

# Generate bar chart for monthly review volume by quarter
fig = px.bar(
    df,
    x='QUARTER', 
    y='Total Reviews', 
    color='Total Reviews', 
    labels={'QUARTER': 'Quarter', 'Total Reviews': 'Review Count'},
    title='Figure 7: Quarterly Yelp Review Volume Trends'
)
fig.update_xaxes(dtick=1)
fig.show()

,QUARTER,ONE_STAR_REVIEWS,TWO_STAR_REVIEWS,THREE_STAR_REVIEWS,FOUR_STAR_REVIEWS,FIVE_STAR_REVIEWS,TOTAL_REVIEWS,TOTAL_STARS,AVG_RATING
0,1,254879,135483,176621,375630,804599,1747212,6581223,3.77
1,2,267952,136048,171950,360521,803312,1739783,6514542,3.74
2,3,288448,145698,182094,379363,860782,1856385,6947488,3.74
3,4,258282,127011,161269,337404,762934,1646900,6160397,3.74


### Quarterly Review Volume & Star Metrics by Year

In [34]:
# Reviews by year and quarter
df = cur.execute('''
                SELECT 
                    YEAR(review_date) AS year,
                    QUARTER(review_date) AS quarter,
                    SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                    SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                    SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                    SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                    SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                    COUNT(*) AS total_reviews,
                    SUM(stars) AS total_stars,
                    ROUND(AVG(stars), 2) avg_rating
                FROM yelp_reviews
                GROUP BY 1, 2
                ORDER BY 1, 2;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 17: Monthly Review Volume & Star Metrics</span>').set_table_styles(styles))

# Rename columns for readability in charts
df.rename(columns={'ONE_STAR_REVIEWS': '1 Star', 'TWO_STAR_REVIEWS': '2 Star', 'THREE_STAR_REVIEWS': '3 Star', 'FOUR_STAR_REVIEWS': '4 Star', 'FIVE_STAR_REVIEWS': '5 Star', 'TOTAL_REVIEWS': 'Total Reviews',
                   'AVG_RATING': 'Average Rating'}, inplace=True)

# Create a date column from year and quarter
df['date'] = 'Q' + df['QUARTER'].astype(str) + '-' + df['YEAR'].astype(str)

# Melt into long format for counts of reviews with each star value
long_df = df.melt(
    id_vars=['date'],
    value_vars=['1 Star','2 Star','3 Star','4 Star','5 Star'],
    var_name='star',
    value_name='count'
)

# Plot time series of star value counts by year and quarter
fig = px.line(
    long_df,
    x='date',
    y='count',
    color='star',
    labels={
        'date':'Quarter-Year',
        'count':'Review Count',
        'star':'Star Rating'
    },
    title='Figure 8: Quarterly Yelp Star Rating Counts by Year'
)
fig.show()

# Melt into long format for total review counts for each quarter
long_df = df.melt(
    id_vars=['date'],
    value_vars=['Total Reviews'],
    var_name='reviews',
    value_name='count'
)

# Plot time series of total reviews by year and quarter
fig = px.line(
    long_df, 
    x='date', 
    y='count', 
    labels={
        'date': 'Quarter-Year',
        'count': 'Review Count'
        }, 
    title='Figure 9: Quarterly Yelp Review Volume by Year'
)
fig.show()

# Melt into long format for average star rating for each quarter
long_df = df.melt(
    id_vars=['date'],
    value_vars=['Average Rating'],
    var_name='avg_stars',
    value_name='count'
)

# Plot time series of average star rating by year and quarter
fig = px.line(
    long_df, 
    x='date', 
    y='count', 
    labels={
        'date': 'Quarter-Year',
        'count': 'Average Star Rating'
        }, 
    title='Figure 10: Quarterly Average Yelp Star Rating by Year'
)
fig.show()

,YEAR,QUARTER,ONE_STAR_REVIEWS,TWO_STAR_REVIEWS,THREE_STAR_REVIEWS,FOUR_STAR_REVIEWS,FIVE_STAR_REVIEWS,TOTAL_REVIEWS,TOTAL_STARS,AVG_RATING
0,2005,1,5,8,18,22,24,77,283,3.68
1,2005,2,4,9,24,67,68,172,702,4.08
2,2005,3,15,26,97,144,112,394,1494,3.79
3,2005,4,6,11,32,82,80,211,852,4.04
4,2006,1,23,27,88,217,242,597,2419,4.05
64,2021,1,26701,9094,9298,17966,79443,142502,541862,3.80
65,2021,2,34641,11786,12022,21433,89507,169389,627546,3.70
66,2021,3,34968,11705,12006,21099,83513,163291,596357,3.65
67,2021,4,30140,9909,10361,19565,73032,143007,524461,3.67
68,2022,1,6607,1959,2183,4351,16565,31665,117303,3.70


## Geographic Breakdown

### Number of Business and Average Star Rating by State

In [36]:
# Number of businesses and avg star rating by state
df = cur.execute('''
                SELECT 
                    state,
                    COUNT(DISTINCT yb.business_id) num_businesses,
                    ROUND(AVG(yr.stars), 2) as avg_rating
                FROM yelp_reviews yr
                JOIN yelp_businesses yb
                    ON yr.business_id = yb.business_id
                GROUP BY state
                ORDER BY 2 DESC;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 18: Business Volume by State</span>').set_table_styles(styles))

,STATE,NUM_BUSINESSES,AVG_RATING
0,PA,34039,3.70
1,FL,26330,3.76
2,TN,12056,3.76
3,IN,11247,3.79
4,MO,10913,3.73
5,LA,9924,3.88
6,AZ,9912,3.69
7,NJ,8536,3.56
8,NV,7715,3.73
9,AB,5573,3.57


### Number of Businesses and Average Star Rating by City

In [35]:
# Number of businesses and avg star rating by city
df = cur.execute('''
                SELECT 
                    city,
                    COUNT(DISTINCT yb.business_id) num_businesses,
                    ROUND(AVG(yr.stars), 2) as avg_rating
                FROM yelp_reviews yr
                JOIN yelp_businesses yb
                    ON yr.business_id = yb.business_id
                GROUP BY city
                ORDER BY 2 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 19: Business Volume by City</span>').set_table_styles(styles))

,CITY,NUM_BUSINESSES,AVG_RATING
0,Philadelphia,14569,3.79
1,Tucson,9250,3.70
2,Tampa,9050,3.75
3,Indianapolis,7540,3.82
4,Nashville,6971,3.82
5,New Orleans,6209,3.94
6,Reno,5935,3.75
7,Edmonton,5054,3.57
8,Saint Louis,4827,3.81
9,Santa Barbara,3829,4.00


## Business-Level Insight

### Most Recent Reviews for Each Business

In [38]:
# Most recent reviews for each business
df = cur.execute('''
                WITH cte AS (
                    SELECT 
                        business_id,
                        review_date,
                        user_review,
                        stars,
                        ROW_NUMBER() OVER(PARTITION BY business_id ORDER BY review_date DESC) AS rn
                    FROM yelp_reviews
                )
                SELECT 
                    yb.name,
                    yb.city,
                    yb.state,
                    yb.postal_code,
                    review_date,
                    cte.stars,
                    user_review
                FROM cte
                JOIN yelp_businesses yb
                    ON cte.business_id = yb.business_id
                WHERE rn=1;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(3), df.tail(3)])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 20: Most Recent Reviews for Each Business</span>').set_table_styles(styles))

,NAME,CITY,STATE,POSTAL_CODE,REVIEW_DATE,STARS,USER_REVIEW
0,Webster Groves Public Library,Webster Groves,MO,63119,2019-08-24,5,"I've been going to this library ever since I learned how to read as a kid. The staff has always been extremely friendly and helpful. It looks even more beautiful with the update a few years ago. This library has a wondering children's section, pretty big YA section, and all the adult fiction and nonfiction you could ever ask for. You can also request books online from other libraries around STL, as well as get ebooks and audiobooks from Overdrive/Libby and Hoopla. This library has it all! There are also lots of flyers to keep you up to date on what's going on in the neighborhood."
1,Starbucks,Philadelphia,PA,19154,2019-04-25,1,"Horrid Starbucks. They refuse to make a drink if they are too busy. If the drink is on the menu, it should always be available, not just when you feel like making it. (The drink is a plain decaf coffee - not something complicated.)"
2,Soundfactor Entertainment & Events,Boyertown,PA,19512,2021-07-23,5,"Soundfactor was absolutely awesome to work with from start to finish. Having to postpone our wedding during the pandemic was stressful and some vendors were more difficult to deal with than others, but Soundfactor was so flexible and understanding. It alleviated some of our stress knowing that we had one less thing to worry about having our DJ secured for our new date. Their website was so organized and helped us to put in all of our requests and create our wedding timeline. Steve, our DJ, called us to discuss all the details and make sure that he knew everything we wanted so that he could make our day perfect! And it did turn out perfect! Steve was fantastic. He knew how to read the room and play what the crowd was enjoying while also sticking to a lot of our requested songs. Our dance floor was packed the entire night. Steve had everyone on the dance floor from our 25 year old friends to my 87 year old grandmother (who I haven't seen dance in years!). We got so many compliments on the music and how much fun our wedding was!! I highly recommend Soundfactor for anyone looking to have a perfect wedding reception."
150343,FabuLASHby Michelle,Reno,NV,89503,2013-07-05,1,"I had extensions for over 2 years and moved out of state and found Michelle online, thought her prices were reasonable and ratings good so I booked my appointment. Michelle is a very nice gal and very accommodating but she used so much glue and loaded so many extensions on I'd have sometimes up to 4 extensions per lash, which in turn DESTROYED my natural lash making them shorter, significantly thinner, and so weak a fill wouldn't hardly last a week. I went back home for a visit and went to my prior lady and she recommended not adding any since my natural lashes were stripped and gone and hadn't seen lashes look so poor. I saw Michelle about 5 times and had also gave her the name of my prior lady to contact, as she was interested in the methods, brands, glue etc. she used. Now I'm left using lash serum to grow them and make them healthy again."
150344,WOW! Factor Desserts,Sherwood Park,AB,T8A 3X8,2019-07-20,4,"I was introduced to this place after I interrogated my coworker about where she got this amazing cake while at a staff party. That was almost 10 years ago and based on how I was impacted by this sliced piece of perfection, I wanted a cake for my Birthday. My wife got this giant 4 layer chocolate cake that took me to a special place of pure bliss. I would give it a perfect score but there were a few notes to be aware: 1. Hours of operation are really limited where they are closed on the weekends and outside work hours, making it hard to access. 2. Cakes come frozen, so you have to give at least 4 hours to thaw to be ready to serve. 3. Service was reasonable but not exceptional. It was like they didn't feel the need to entice our business and did bare minimum. 4. Cakes were enormous, one standard size, and intende

### Most Reviewed Businesses and Their Average Star Rating

In [39]:
# Most reviewed businesses
df = cur.execute('''
                SELECT yb.name, yb.city, yb.state, COUNT(yr.review_id) AS review_count, ROUND(AVG(yr.stars), 2) AS avg_rating
                FROM yelp_reviews yr
                JOIN yelp_businesses yb
                ON yr.business_id = yb.business_id
                GROUP BY yb.name, yb.city, yb.state
                ORDER BY COUNT(yr.review_id) DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 21: Most Reviewed Businesses</span>').set_table_styles(styles))

,NAME,CITY,STATE,REVIEW_COUNT,AVG_RATING
0,Acme Oyster House,New Orleans,LA,7673,4.12
1,Oceana Grill,New Orleans,LA,7516,4.15
2,Hattie B’s Hot Chicken - Nashville,Nashville,TN,6160,4.45
3,Reading Terminal Market,Philadelphia,PA,5778,4.61
4,Ruby Slipper Cafe,New Orleans,LA,5523,4.21
5,Ruby Slipper - New Orleans,New Orleans,LA,5264,4.29
6,Mother's Restaurant,New Orleans,LA,5254,3.44
7,Royal House,New Orleans,LA,5146,3.79
8,Commander's Palace,New Orleans,LA,4969,4.29
9,Los Agaves,Santa Barbara,CA,4718,4.44


### Top Businesses by State

In [41]:
# Highest rated businesses for the selected state by average stars and total stars
df = cur.execute('''
                WITH ranked AS (
                SELECT
                    yb.name,
                    yb.state,
                    ROUND(AVG(yr.stars),2) AS avg_rating,
                    COUNT(yr.review_id) AS total_reviews
                FROM yelp_businesses yb
                JOIN yelp_reviews yr
                    ON yb.business_id = yr.business_id
                WHERE yb.state = 'IL'
                GROUP BY 1,2
                )
                SELECT
                    name,
                    state,
                    avg_rating,
                    total_reviews,
                    avg_rating * total_reviews AS total_stars
                FROM ranked
                ORDER BY avg_rating DESC, total_reviews DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 22: Highest Rated Businesses in the Selected State</span>').set_table_styles(styles))

,NAME,STATE,AVG_RATING,TOTAL_REVIEWS,TOTAL_STARS
0,Grit & Tonic Handcrafted,IL,5.00,22,110.00
1,CP Pinball,IL,5.00,18,90.00
2,Walton's Ice Cream and More,IL,5.00,13,65.00
3,"Michael Murphy, OD",IL,5.00,12,60.00
4,Mike's iPhone Repair Service,IL,5.00,12,60.00
5,CBC Home Inspections,IL,5.00,12,60.00
6,Post Pack & Ship,IL,5.00,11,55.00
7,Eaker's Family Barber Shop,IL,5.00,11,55.00
8,Jacks 66 Auto Repair,IL,5.00,10,50.00
9,Exactime Watch & Clock,IL,5.00,10,50.00


### Top Businesses by City

In [42]:
# Highest rated businesses for the selected city by average stars and total stars
df = cur.execute('''
                WITH ranked AS (
                SELECT
                    yb.name,
                    yb.city,
                    yb.state,
                    ROUND(AVG(yr.stars),2) AS avg_rating,
                    COUNT(yr.review_id) AS total_reviews
                FROM yelp_businesses yb
                JOIN yelp_reviews yr
                    ON yb.business_id = yr.business_id
                WHERE yb.city = 'New Orleans'
                GROUP BY 1, 2, 3
                )
                SELECT
                    name,
                    city,
                    state,
                    avg_rating,
                    total_reviews,
                    avg_rating * total_reviews AS total_stars
                FROM ranked
                ORDER BY avg_rating DESC, total_reviews DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 23: Highest Rated Businesses in the Selected City</span>').set_table_styles(styles))

,NAME,CITY,STATE,AVG_RATING,TOTAL_REVIEWS,TOTAL_STARS
0,Drink & Learn,New Orleans,LA,5.00,90,450.00
1,New Orleans Streetwalkers Tours,New Orleans,LA,5.00,52,260.00
2,Hidden History Tours,New Orleans,LA,5.00,42,210.00
3,Casa Pelican B&B and Cooking School,New Orleans,LA,5.00,42,210.00
4,Slate Detail,New Orleans,LA,5.00,41,205.00
5,RachelLauren Massage,New Orleans,LA,5.00,36,180.00
6,Terrell House,New Orleans,LA,5.00,35,175.00
7,New Orleans Architecture Tours,New Orleans,LA,5.00,34,170.00
8,WobbeMassage,New Orleans,LA,5.00,31,155.00
9,Eversaint Salon,New Orleans,LA,5.00,30,150.00


### Average Star Rating: Closed vs. Open Businesses

In [43]:
# Average rating for closed vs currently operating businesses
df = cur.execute('''
                SELECT 
                    'Open' AS business_status, 
                    ROUND(AVG(yr.stars), 2) as avg_rating
                FROM yelp_businesses yb
                JOIN yelp_reviews yr
                    ON yb.business_id = yr.business_id
                WHERE is_open = 1
                UNION
                SELECT 
                    'Closed',
                    ROUND(AVG(yr.stars), 2) as avg_rating
                FROM yelp_businesses yb
                JOIN yelp_reviews yr
                    ON yb.business_id = yr.business_id
                WHERE is_open = 0;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 24: Average Rating: Closed vs. Open Businesses</span>').set_table_styles(styles))

,BUSINESS_STATUS,AVG_RATING
0,Open,3.77
1,Closed,3.62


### Average Star Rating by Category: Closed vs. Open Businesses

In [44]:
# Average rating for closed vs currently operating businesses by category
df = cur.execute('''
                WITH status AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT 
                        business_id,
                        TRIM(A.value) AS category,
                        CASE
                            WHEN is_open = 1 THEN 'open'
                            ELSE 'closed'
                        END AS business_status
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )
                SELECT 
                    category,
                    ROUND(AVG(CASE WHEN business_status = 'open' THEN yr.stars END), 2) as avg_open_rating,
                    ROUND(AVG(CASE WHEN business_status = 'closed' THEN yr.stars END), 2) as avg_closed_rating
                FROM status
                JOIN yelp_reviews yr
                    ON status.business_id = yr.business_id
                GROUP BY category;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.head(10).style.set_caption('<span style="font-size: 20px;">Table 25: Average Star Rating by Category: Closed vs. Open Businesses</span>').set_table_styles(styles))

print(df.info())

open_higher = df[df['AVG_OPEN_RATING'] > df['AVG_CLOSED_RATING']].shape[0]
closed_higher = df[df['AVG_OPEN_RATING'] <= df['AVG_CLOSED_RATING']].shape[0]

print(f'Categories with avg_open_RATING greater than avg_closed_rating: {open_higher}\nCategories with avg_open_rating less than or equal to avg_closed_rating: {closed_higher}')

print(f'''
      Out of the 1311 categories with at least one review for both closed and open businesses, there are {open_higher} categories where the average
      star rating for open businesses is greater than closed businesses, and {closed_higher} categories where the average star rating for open 
      businesses is less than or equal to closed businesses.
      ''')

,CATEGORY,AVG_OPEN_RATING,AVG_CLOSED_RATING
0,Restaurants,3.83,3.64
1,Candy Stores,4.33,3.93
2,Specialty Food,4.05,3.90
3,Day Spas,3.87,3.61
4,American (Traditional),3.71,3.49
5,Modern European,4.18,3.95
6,Sports Bars,3.42,3.19
7,Cosmetic Dentists,3.75,3.08
8,Professional Services,3.55,3.19
9,Arcades,3.37,3.52


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CATEGORY           1311 non-null   object
 1   AVG_OPEN_RATING    1302 non-null   object
 2   AVG_CLOSED_RATING  1048 non-null   object
dtypes: object(3)
memory usage: 30.9+ KB
None
Categories with avg_open_RATING greater than avg_closed_rating: 715
Categories with avg_open_rating less than or equal to avg_closed_rating: 324

      Out of the 1311 categories with at least one review for both closed and open businesses, there are 715 categories where the average
      star rating for open businesses is greater than closed businesses, and 324 categories where the average star rating for open 
      businesses is less than or equal to closed businesses.
      


### Businesses with Lowest Average Star Rating (Min 100 Reviews) 

In [45]:
# Businesses with lowest average stars (min 100 reviews)
df = cur.execute('''
                SELECT 
                    name,
                    city,
                    state,
                    COUNT(yr.review_id) AS review_count,
                    ROUND(AVG(yr.stars), 2) AS avg_rating 
                FROM yelp_businesses yb
                JOIN yelp_reviews yr
                    ON yb.business_id = yr.business_id
                GROUP BY name, city, state
                HAVING COUNT(yr.review_id) >= 100
                ORDER BY AVG(yr.stars), COUNT(yr.review_id) DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 26: Lowest Average Rating by Business (Min 100 Reviews)</span>').set_table_styles(styles))

,NAME,CITY,STATE,REVIEW_COUNT,AVG_RATING
0,First Advantage Corporation,Saint Petersburg,FL,103,1.00
1,717 Parking Enterprises,Tampa,FL,106,1.04
2,International Medical Group,Indianapolis,IN,168,1.04
3,PHH Mortgage,Mount Laurel,NJ,167,1.04
4,Sensible Home Warranty,Sparks,NV,175,1.05
5,Express Scripts,Maryland Heights,MO,318,1.06
6,Sears Home Services,Fenton,MO,587,1.08
7,Defender Security Company,Indianapolis,IN,428,1.08
8,Penrose Hotel,Philadelphia,PA,152,1.09
9,EXPRESS SCRIPTS,St. Louis,MO,224,1.09


## Business-Specific Deep Dive

### Weekly Review Count & Week-over-Week Change

In [47]:
# Weekly review count and week-to-week change for Reading Terminal Market in Philadelphia, PA
df = cur.execute('''
                WITH weekly_reviews AS (
                    SELECT 
                        DATE_TRUNC('week', review_date) AS wk,
                        COUNT(*) AS reviews_this_week
                    FROM yelp_reviews
                    WHERE business_id = 'ytynqOUb3hjKeJfRj5Tshw'
                    GROUP BY 1
                )
                SELECT 
                    wk AS week,
                    reviews_this_week,
                    reviews_this_week - LAG(reviews_this_week) OVER(ORDER BY wk) AS review_vol_change
                FROM weekly_reviews
                ORDER BY wk;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 27: Weekly Review Count & Week-over-Week Change</span>').set_table_styles(styles))

,WEEK,REVIEWS_THIS_WEEK,REVIEW_VOL_CHANGE
0,2005-11-21,1,None
1,2006-06-05,1,0
2,2006-10-23,1,0
3,2006-11-13,1,0
4,2006-12-04,2,1
729,2021-12-20,4,1
730,2021-12-27,7,3
731,2022-01-03,10,3
732,2022-01-10,11,1
733,2022-01-17,5,-6


### Rolling Star Rating Average (Last 10 Reviews)

In [48]:
# Rolling average star rating over last 10 reviews
df = cur.execute('''
                SELECT 
                    review_date,
                    stars,
                    ROUND(AVG(stars)
                        OVER(ORDER BY review_date
                        ROWS BETWEEN 9 PRECEDING AND CURRENT ROW), 2) AS last_10_reviews_avg
                FROM yelp_reviews
                WHERE business_id = 'ytynqOUb3hjKeJfRj5Tshw'
                ORDER BY review_date;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 28: Rolling Star Rating Average (Last 10 Reviews)</span>').set_table_styles(styles))

,REVIEW_DATE,STARS,LAST_10_REVIEWS_AVG
0,2005-11-27,5,5.00
1,2006-06-05,5,5.00
2,2006-10-24,5,5.00
3,2006-11-13,5,5.00
4,2006-12-04,5,5.00
5773,2022-01-17,5,4.90
5774,2022-01-18,5,4.90
5775,2022-01-19,4,4.80
5776,2022-01-19,5,4.90
5777,2022-01-19,5,4.90


### Cumulative Average & Review Sequence

In [49]:
# Cumulative average and review count
df = cur.execute('''
                SELECT 
                    review_date,
                    stars,
                    ROUND(AVG(stars)
                        OVER(ORDER BY review_date
                        ROWS UNBOUNDED PRECEDING), 3) AS cumulative_avg_rating,
                    ROW_NUMBER() OVER(ORDER BY review_date) AS cumulative_review_count
                FROM yelp_reviews
                WHERE business_id = 'ytynqOUb3hjKeJfRj5Tshw'
                ORDER BY review_date;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 29: Cumulative Average & Review Count</span>').set_table_styles(styles))

,REVIEW_DATE,STARS,CUMULATIVE_AVG_RATING,CUMULATIVE_REVIEW_COUNT
0,2005-11-27,5,5.000,1
1,2006-06-05,5,5.000,2
2,2006-10-24,5,5.000,3
3,2006-11-13,5,5.000,4
4,2006-12-04,5,5.000,6
5773,2022-01-17,5,4.605,5774
5774,2022-01-18,5,4.605,5775
5775,2022-01-19,5,4.605,5776
5776,2022-01-19,5,4.605,5777
5777,2022-01-19,4,4.605,5778


### Monthly Star Rating Distribution by Year

In [50]:
# Star rating distribution by month
df = cur.execute('''
                WITH monthly_stars AS (
                    SELECT 
                        DATE_TRUNC('month', review_date) AS mnth,
                        SUM(CASE WHEN stars = 5 THEN 1 ELSE 0 END) AS five_star_count,
                        SUM(CASE WHEN stars = 4 THEN 1 ELSE 0 END) AS four_star_count,
                        SUM(CASE WHEN stars = 3 THEN 1 ELSE 0 END) AS three_star_count,
                        SUM(CASE WHEN stars = 2 THEN 1 ELSE 0 END) AS two_star_count,
                        SUM(CASE WHEN stars = 1 THEN 1 ELSE 0 END) AS one_star_count,
                        COUNT(*) AS total_reviews
                    FROM yelp_reviews
                    WHERE business_id = 'ytynqOUb3hjKeJfRj5Tshw'
                    GROUP BY 1
                )
                SELECT 
                    mnth as month,
                    total_reviews,
                    ROUND(five_star_count::NUMERIC / total_reviews * 100, 2) AS pct_five_star,
                    ROUND(four_star_count::NUMERIC / total_reviews * 100, 2) AS pct_four_star,
                    ROUND(three_star_count::NUMERIC / total_reviews * 100, 2) AS pct_three_star,
                    ROUND(two_star_count::NUMERIC / total_reviews * 100, 2) AS pct_two_star,
                    ROUND(one_star_count::NUMERIC / total_reviews * 100, 2) AS pct_one_star
                FROM monthly_stars
                ORDER BY mnth;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 30: Monthly Star Rating Distribution by Year</span>').set_table_styles(styles))

,MONTH,TOTAL_REVIEWS,PCT_FIVE_STAR,PCT_FOUR_STAR,PCT_THREE_STAR,PCT_TWO_STAR,PCT_ONE_STAR
0,2005-11-01,1,100.00,0.00,0.00,0.00,0.00
1,2006-06-01,1,100.00,0.00,0.00,0.00,0.00
2,2006-10-01,1,100.00,0.00,0.00,0.00,0.00
3,2006-11-01,1,100.00,0.00,0.00,0.00,0.00
4,2006-12-01,5,100.00,0.00,0.00,0.00,0.00
181,2021-09-01,19,84.21,15.79,0.00,0.00,0.00
182,2021-10-01,35,77.14,17.14,5.71,0.00,0.00
183,2021-11-01,20,70.00,10.00,15.00,5.00,0.00
184,2021-12-01,22,95.45,0.00,4.55,0.00,0.00
185,2022-01-01,30,86.67,13.33,0.00,0.00,0.00


### Reviewer Bias: Business Rating vs. Reviewer's Overall Average Star Rating

In [51]:
# Difference between reviewer's overall average rating and their rating for the business
df = cur.execute('''
                WITH business_reviews AS (
                    SELECT 
                        user_id,
                        stars AS business_rating
                    FROM yelp_reviews
                    WHERE business_id = 'ytynqOUb3hjKeJfRj5Tshw'
                )
                SELECT 
                    br.user_id,
                    business_rating,
                    yu.avg_stars,
                    business_rating - yu.avg_stars AS rating_diff
                FROM business_reviews br
                JOIN yelp_users yu
                    ON br.user_id = yu.user_id;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 31: Reviewer Bias</span>').set_table_styles(styles))

,USER_ID,BUSINESS_RATING,AVG_STARS,RATING_DIFF
0,qVc8ODYU5SZjKXVBgXdI7w,5,3.910000,1.090000
1,j14WgRoU_-2ZE1aw1dXrJg,4,3.740000,0.260000
2,OJorsQjotS1dIv--IVifCA,5,3.770000,1.230000
3,I7YQFiIsO82eliq8nqsLfA,5,3.840000,1.160000
4,ITa3vh5ERI90G_WP4SmGUQ,5,4.010000,0.990000
5773,tOMF_9TvCMsRu9k1FxZ9KA,4,3.880000,0.120000
5774,xZBhrOt7A3u8UGUyXFsAcQ,5,4.540000,0.460000
5775,KJ25vVzMsWGdzUDoeHImIQ,5,4.800000,0.200000
5776,LIC3CgJuzEjbTmSY7bicXQ,5,4.330000,0.670000
5777,8BCcPyubwAqHjllpnGsVuw,2,3.530000,-1.530000


### Time Between Reviews

In [53]:
# Time between reviews in days (can be used to find average days between review for a specified time period)
df = cur.execute('''
                SELECT 
                    review_date,
                    LAG(review_date) OVER(ORDER BY review_date) AS previous_date,
                    review_date - LAG(review_date) OVER(ORDER BY review_date) AS days_since_last_review
                FROM yelp_reviews
                WHERE business_id = 'ytynqOUb3hjKeJfRj5Tshw'
                ORDER BY review_date;
                 ''').fetch_pandas_all()

df_preview = pd.concat([df.head(), df.tail()])
styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df_preview.style.set_caption('<span style="font-size: 20px;">Table 32: Time Between Reviews (Days)</span>').set_table_styles(styles))

,REVIEW_DATE,PREVIOUS_DATE,DAYS_SINCE_LAST_REVIEW
0,2005-11-27,None,nan
1,2006-06-05,2005-11-27,190.000000
2,2006-10-24,2006-06-05,141.000000
3,2006-11-13,2006-10-24,20.000000
4,2006-12-04,2006-12-04,0.000000
5773,2022-01-17,2022-01-16,1.000000
5774,2022-01-18,2022-01-17,1.000000
5775,2022-01-19,2022-01-18,1.000000
5776,2022-01-19,2022-01-19,0.000000
5777,2022-01-19,2022-01-19,0.000000


## User Engagement Funnel & Top Contributors

### User Engagement Funnel

In [54]:
# Create user engagement funnel
df = cur.execute('''
                SELECT 'Registered Users' AS User_Type, COUNT(*) AS Number_Of_Users, 100 AS Pct_Of_Users
                FROM yelp_users
                UNION
                SELECT 'Reviewers', COUNT(user_id), ROUND(COUNT(user_id) / (SELECT COUNT(*) FROM yelp_users) * 100, 3)
                FROM yelp_users
                WHERE review_count >= 1
                UNION
                SELECT 'Highly Active Reviewers', COUNT(user_id), ROUND(COUNT(user_id) / (SELECT COUNT(*) FROM yelp_users) * 100, 3)
                FROM yelp_users
                WHERE review_count >= (
                SELECT ROUND(AVG(review_count), 2) AS avg_reviews
                FROM yelp_users
                )
                UNION
                SELECT 'Tippers', COUNT(DISTINCT user_id), ROUND(COUNT(DISTINCT user_id) / (SELECT COUNT(*) FROM yelp_users) * 100, 3)
                FROM yelp_tips
                UNION
                SELECT 'Influential Users', COUNT(user_id), ROUND(COUNT(user_id) / (SELECT COUNT(*) FROM yelp_users) * 100, 3)
                FROM yelp_users
                WHERE fans >= (
                SELECT ROUND(AVG(fans), 2) AS avg_fans
                FROM yelp_users
                )
                UNION
                SELECT 'Elite Users', COUNT(user_id), ROUND(COUNT(user_id) / (SELECT COUNT(*) FROM yelp_users) * 100, 3)
                FROM yelp_users
                WHERE elite != ''
                UNION
                SELECT 'New Users', COUNT(user_id), ROUND(COUNT(user_id) / (SELECT COUNT(*) FROM yelp_users) * 100, 3)
                FROM yelp_users
                WHERE EXTRACT(year from yelping_since) >= 2021
                ORDER BY 2 DESC
                ;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 33: User Engagement Funnel</span>').set_table_styles(styles))

fig = go.Figure(go.Funnel(
    y=df['USER_TYPE'].values,
    x=df['NUMBER_OF_USERS'].values,
    textinfo='value+percent initial',
    marker= {
        'color': ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b', '#e377c2']
        },
    textfont={'color': 'black', 'size': 14},
    textposition='outside'
    )
)
fig.update_layout(title_text='Figure 11: User Engagement Funnel')
fig.show()


,USER_TYPE,NUMBER_OF_USERS,PCT_OF_USERS
0,Registered Users,1987897,100.000
1,Reviewers,1987843,99.997
2,Highly Active Reviewers,374650,18.847
3,Tippers,301758,15.180
4,Influential Users,210208,10.574
5,Elite Users,91198,4.588
6,New Users,43267,2.177


### Most Active Reviewers

In [55]:
# Users with the most reviews
df = cur.execute('''
                SELECT
                    yu.user_id,
                    yu.fans,
                    COUNT(yr.review_id) AS reviews_written,
                    ROUND(AVG(yr.stars), 2) AS avg_rating,
                    SUM(yr.useful_votes) AS total_useful_votes,
                    ROUND(AVG(yr.useful_votes), 2) AS avg_useful_votes
                FROM yelp_users yu
                JOIN yelp_reviews yr
                    ON yu.user_id = yr.user_id
                GROUP BY yu.user_id, yu.fans
                ORDER BY COUNT(yr.review_id) DESC
                LIMIT 15;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 34: Most Active Reviewers</span>').set_table_styles(styles))

,USER_ID,FANS,REVIEWS_WRITTEN,AVG_RATING,TOTAL_USEFUL_VOTES,AVG_USEFUL_VOTES
0,_BcWyKQL16ndpBdggh2kNA,558,3048,3.64,11043,3.62
1,Xw7ZjaGfr0WNVt6s_5KZfA,379,1840,4.07,12011,6.53
2,0Igx-a1wAstiBDerGxXk2A,200,1747,3.99,8254,4.72
3,-G7Zkl1wIWBBmD0KRy_sCw,703,1682,3.65,34694,20.63
4,ET8n-r7glWYqZhuR6GcdNw,1353,1653,4.05,19474,11.78
5,bYENop4BuQepBjM1-BI3fA,401,1578,3.85,11680,7.40
6,1HM81n6n4iPIFU5d2Lokhw,149,1554,3.05,6138,3.95
7,fr1Hz2acAb3OaL3l6DyKNg,387,1447,3.95,15356,10.61
8,wXdbkFZsfDR7utJvbWElyA,250,1396,4.21,5715,4.09
9,Um5bfs5DH6eizgjH3xZsvg,203,1391,3.80,4645,3.34


### Most Helpful Reviewers (Min 100 Reviews)

In [56]:
# Users with the most useful votes per review (min 100 reviews)
df = cur.execute('''
                SELECT
                    yu.user_id,
                    yu.fans,
                    COUNT(yr.review_id) AS reviews_written,
                    ROUND(AVG(yr.stars), 2) AS avg_rating,
                    SUM(yr.useful_votes) AS total_useful_votes,
                    ROUND(AVG(yr.useful_votes), 2) AS avg_useful_votes
                FROM yelp_users yu
                JOIN yelp_reviews yr
                    ON yu.user_id = yr.user_id
                GROUP BY yu.user_id, yu.fans
                HAVING COUNT(yr.review_id) >= 100
                ORDER BY ROUND(AVG(yr.useful_votes), 2) DESC
                LIMIT 15;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 35: Most Helpful Reviewers (Min 100 Reviews)</span>').set_table_styles(styles))

,USER_ID,FANS,REVIEWS_WRITTEN,AVG_RATING,TOTAL_USEFUL_VOTES,AVG_USEFUL_VOTES
0,tsMF0FcFcHZ8i28WzWtQXw,391,218,3.83,15112,69.32
1,A4bsa7ykYRVCnb4h2vZALw,737,298,4.08,17702,59.40
2,YTu0c-1I9ECeksIgoHXFPA,558,120,4.53,5290,44.08
3,6jjHo9Lilv3kTy87pm2ycw,600,273,4.28,11476,42.04
4,Hm0diOkWwpo9zotlJlqMUQ,634,103,4.17,4223,41.00
5,wzUFZ3DaU7k9R2S54IMPfg,174,402,4.38,13400,33.33
6,a4mO387_O77ebmv8AlQB3A,539,131,3.97,3449,26.33
7,BmVwbsL8l0imz4slonyMaA,713,250,4.37,6097,24.39
8,lMY8NBPyzlPbbu-KBYfD9A,740,744,4.13,17358,23.33
9,4DyMRBKrokax6d8LCI3GAw,563,228,4.66,5298,23.24


## Comparative User Metrics

### Review Metrics: Elite vs. Non-Elite Users

In [57]:
# Comparing elite and non-elite user reviews
df = cur.execute('''
                WITH review_stats AS (
                    SELECT 
                        IFF(LENGTH(elite) > 0, 'Elite', 'Regular') AS user_status,
                        yr.stars,
                        yr.useful_votes,
                        yr.funny_votes,
                        yr.cool_votes,
                        yr.user_review
                    FROM yelp_users yu
                    JOIN yelp_reviews yr
                        ON yu.user_id = yr.user_id
                )
                SELECT 
                    user_status,
                    ROUND(AVG(stars), 2) AS avg_rating,
                    ROUND(AVG(useful_votes), 2) AS avg_useful_votes,
                    ROUND(AVG(funny_votes), 2) AS avg_funny_votes,
                    ROUND(AVG(cool_votes), 2) AS avg_cool_votes,
                    ROUND(AVG(LENGTH(user_review)), 2) AS avg_review_length,
                    COUNT(*) AS total_reviews
                FROM review_stats
                GROUP BY user_status;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 36: Elite vs. Non-Elite User Review Metrics</span>').set_table_styles(styles))

,USER_STATUS,AVG_RATING,AVG_USEFUL_VOTES,AVG_FUNNY_VOTES,AVG_COOL_VOTES,AVG_REVIEW_LENGTH,TOTAL_REVIEWS
0,Regular,3.68,0.82,0.18,0.21,498.78,5264589
1,Elite,3.97,2.30,0.76,1.38,778.23,1725658


### Review Engagement & Metrics by Star Rating

In [58]:
# Comparing engagement on reviews of each star value
df = cur.execute('''
                WITH review_stats AS (
                    SELECT 
                        CASE
                            WHEN stars = 1 THEN '1 Star'
                            WHEN stars = 2 THEN '2 Star'
                            WHEN stars = 3 THEN '3 Star'
                            WHEN stars = 4 THEN '4 Star'
                            ELSE '5 Star'
                        END AS rating,
                        yr.stars,
                        yr.useful_votes,
                        yr.funny_votes,
                        yr.cool_votes,
                        yr.user_review
                    FROM yelp_reviews yr
                )
                SELECT 
                    rating,
                    ROUND(AVG(useful_votes), 2) AS avg_useful_votes,
                    ROUND(AVG(funny_votes), 2) AS avg_funny_votes,
                    ROUND(AVG(cool_votes), 2) AS avg_cool_votes,
                    COUNT(*) AS total_reviews,
                    ROUND(AVG(LENGTH(user_review)), 2) AS avg_review_length
                FROM review_stats
                GROUP BY rating
                ORDER BY rating
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 37: Review Metrics by Star Rating</span>').set_table_styles(styles))

,RATING,AVG_USEFUL_VOTES,AVG_FUNNY_VOTES,AVG_COOL_VOTES,TOTAL_REVIEWS,AVG_REVIEW_LENGTH
0,1 Star,1.67,0.42,0.15,1069561,713.78
1,2 Star,1.35,0.43,0.26,544240,721.08
2,3 Star,1.18,0.40,0.47,691934,669.20
3,4 Star,1.23,0.38,0.74,1452918,587.50
4,5 Star,0.97,0.24,0.55,3231627,463.02


### Elite-Only Analysis

In [60]:
# Exploring stats for elite users
df = cur.execute('''
                WITH review_stats AS (
                    SELECT 
                        yu.user_id,
                        yu.elite,
                        yr.stars,
                        yr.useful_votes,
                        yr.funny_votes,
                        yr.cool_votes,
                        yr.user_review,
                        yu.fans,
                        yu.yelping_since
                    FROM yelp_users yu
                    JOIN yelp_reviews yr
                        ON yu.user_id = yr.user_id
                    WHERE elite != ''
                )
                SELECT 
                    user_id,
                    elite,
                    fans,
                    ROUND((CURRENT_DATE - yelping_since) / 365, 2) AS account_age_yrs,
                    ROUND(AVG(stars), 2) AS avg_rating,
                    ROUND(AVG(useful_votes), 2) AS avg_useful_votes,
                    ROUND(AVG(funny_votes), 2) AS avg_funny_votes,
                    ROUND(AVG(cool_votes), 2) AS avg_cool_votes,
                    ROUND(AVG(LENGTH(user_review)), 2) AS avg_review_length,
                    COUNT(*) AS total_reviews
                FROM review_stats
                GROUP BY user_id, elite, fans, yelping_since;
                 ''').fetch_pandas_all()

# Handle the unnecessary comma in the middle of 2020 and count the years a user has been elite and drop the column with each elite year
df['ELITE_YEARS'] = df['ELITE'].str.replace('20,20', '2020', regex=True).str.split(',').str.len()

df.drop(labels='ELITE', axis=1, inplace=True)

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.head(15).style.set_caption('<span style="font-size: 20px;">Table 38: Elite Users Metrics</span>').set_table_styles(styles))

# TODO: find if elite status is correlated with any statistics

,USER_ID,FANS,ACCOUNT_AGE_YRS,AVG_RATING,AVG_USEFUL_VOTES,AVG_FUNNY_VOTES,AVG_COOL_VOTES,AVG_REVIEW_LENGTH,TOTAL_REVIEWS,ELITE_YEARS
0,dKwzOOMQte1WbozogRMxmg,4,6.810000,4.45,0.72,0.02,0.26,1042.53,47,2
1,oq6T6FcKl0TA9LV_970_8Q,23,14.320000,4.13,3.09,0.75,1.48,1003.05,153,4
2,SqkbioT3gylpW6h2cV4EGA,19,13.820000,3.83,3.23,0.66,2.44,674.25,126,5
3,XsnL3gdNiPRnj9svFvz-EA,52,13.940000,3.94,3.12,0.90,1.63,1028.30,306,10
4,-FxsSuwDbIII7yo5BjHpiA,48,11.610000,3.98,2.15,0.74,1.51,814.41,399,7
5,qF31VMm3iSwn0eOaUSVBaw,2,8.210000,4.06,0.59,0.06,0.28,632.31,32,1
6,3MpDvy5gEdsbZh9-p92dHg,38,17.730000,3.74,1.38,0.35,0.48,676.68,123,13
7,TR7ezcRznfr1Jif7euZZJg,185,16.550000,3.90,5.60,2.49,3.25,1191.61,406,10
8,VTn42gDRqbhvxIuDz2XAKg,7,13.160000,4.62,0.62,0.22,0.46,1098.27,37,2
9,bTlonpHjT0p7gF8Ex9XlVw,29,15.830000,4.56,1.50,0.56,0.94,558.81,16,10


### Review Engagement & Metrics of Users in the High-fan vs. Low-fan Groups

In [61]:
# Comparing impact of number of fans on review engagement and star rating
df = cur.execute('''
                WITH fan_grouping AS (
                SELECT 
                    yu.user_id,
                    fans,
                    yr.stars,
                    yr.useful_votes,
                    yr.funny_votes,
                    yr.cool_votes,
                    yr.user_review,
                    yr.review_id,
                    CASE
                        WHEN fans > (SELECT MEDIAN(fans) FROM yelp_users) THEN 'High'
                        ELSE 'Low'
                    END AS fan_group
                FROM yelp_users yu
                JOIN yelp_reviews yr
                    ON yu.user_id = yr.user_id
                )
                SELECT 
                    fan_group,
                    ROUND(AVG(stars), 2) AS avg_rating,
                    ROUND(AVG(useful_votes), 2) AS avg_useful_votes,
                    ROUND(AVG(funny_votes), 2) AS avg_funny_votes,
                    ROUND(AVG(cool_votes), 2) AS avg_cool_votes,
                    ROUND(AVG(LENGTH(user_review)), 2) AS avg_review_length,
                    COUNT(*) AS total_reviews
                FROM fan_grouping
                GROUP BY fan_group;
                 ''').fetch_pandas_all()

styles = [{
    'selector': 'caption', 'props': 'text-align: left;'
}]

display(df.style.set_caption('<span style="font-size: 20px;">Table 39: Review Engagement for Fan Groups</span>').set_table_styles(styles))

,FAN_GROUP,AVG_RATING,AVG_USEFUL_VOTES,AVG_FUNNY_VOTES,AVG_COOL_VOTES,AVG_REVIEW_LENGTH,TOTAL_REVIEWS
0,Low,3.62,0.69,0.13,0.14,480.13,3674339
1,High,3.89,1.73,0.54,0.90,664.88,3315908


## Conclusion & Next Steps